01. Initializations

In [7]:
import pandas as pd
import sys
import json
import os
import datetime


root_folder = '../CREME_dataset'
scenario = ["mirai", "disk_wipe", "ransomware", "resource_hijacking", "end_point_dos"]
scenario_folder = ["01_mirai", "02_disk_wipe", "03_ransomware", "04_resource_hijacking", "05_end_point_dos"]
filename = "dataset_generation.log"
label = {'mirai': [1, 2, 3, 5, 9, 11, 12, 13],
         'disk_wipe': [1, 2, 4, 6, 8, 14],
         'ransomware': [1, 2, 4, 6, 7, 12, 15],
         'resource_hijacking': [1, 2, 4, 6, 8, 12, 16],
         'end_point_dos': [1, 2, 4, 8, 10, 12, 17]
         }

ips = {"controller": ["192.168.56.111"],
       "data-logger": ["192.168.56.121"],
       "attacker": ["192.168.56.131"],
       "non-vulnerable": ["192.168.56.141", "192.168.56.142"],
       "vulnerable": ["192.168.56.151"],
       "malicious": ["192.168.56.161"],
       "benign": ["192.168.56.171"],
       "target": ["192.168.56.181"]
       }

Test

In [5]:
df = pd.read_csv(os.path.join(root_folder, "original_label_syslog.csv"))#26940
# df = df[df.HostName=="192.168.56.171"]
# df = df[df.PID_or_IP.notnull()]
# df = df[~df.PID_or_IP.str.contains('^localhost\] [^\[\]]+\[[0-9]+$')]
df.info()
df['EventTemplate'].value_counts(normalize=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235962 entries, 0 to 235961
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Time             235962 non-null  object 
 1   HostName         235962 non-null  object 
 2   Component        209025 non-null  object 
 3   PID_or_IP        235913 non-null  object 
 4   Content          235962 non-null  object 
 5   EventId          235962 non-null  object 
 6   EventTemplate    235962 non-null  object 
 7   ParameterList    235962 non-null  object 
 8   Timestamp        235962 non-null  float64
 9   Label            235962 non-null  int64  
 10  Tactic           235962 non-null  object 
 11  Technique        235962 non-null  object 
 12  SubTechnique     235962 non-null  object 
 13  Label_lifecycle  235962 non-null  int64  
dtypes: float64(1), int64(2), object(11)
memory usage: 25.2+ MB


"GET <*> <*> 200 <*> "-" <*>                                                                                                                                                            0.248349
localhost (<*>[<*>]) - FTP session <*>                                                                                                                                                  0.145227
connect from unknown[<*>]                                                                                                                                                               0.123270
disconnect from unknown[<*>]                                                                                                                                                            0.123270
lost connection after <*> from unknown[<*>]                                                                                                                                             0.123215
localhost (<*>[<*>]) - USER <*> Log

2. Repairing the label

In [118]:
df = pd.read_csv(os.path.join(root_folder, "original_label_syslog.csv"))
del_list = ['Tactic', 'Technique', 'SubTechnique']
result_abs_path = os.path.join(root_folder, "modify")
# print(ips["benign"][0])

tmp_df = df[df.HostName=="192.168.56.171"]
tmp_df = tmp_df[tmp_df.PID_or_IP.notnull()]
tmp_df = tmp_df[tmp_df.PID_or_IP.str.contains('^localhost\] [^\[\]]+\[[0-9]+$')]
tmp_df.Component = tmp_df.PID_or_IP.str.extract('^localhost\] ([^\[\]]+)\[[0-9]+$')
tmp_df.PID_or_IP = tmp_df.PID_or_IP.str.extract('^localhost\] [^\[\]]+\[([0-9]+)$')

df.drop(df[df['HostName']=="192.168.56.171"].index, inplace=True)
df = pd.concat([df, tmp_df])

df.drop(df[df.Component.str.contains('apache-error')].index, inplace=True)

df.Label[df.HostName=="192.168.56.171"] = 0
df.Label_lifecycle[df.HostName=="192.168.56.171"] = 0
df['HostName'] = df['HostName'].replace('192.168.56.171', 'benign-server')
# print(df)
df = df.drop(columns=del_list)

if not os.path.exists(result_abs_path):
    os.makedirs(result_abs_path)
df.to_csv(os.path.join(result_abs_path, "original_relabel_syslog.csv"), index=False)

/var/folders/hg/c084_sjj0_1dhg3qjvswf_mh0000gn/T/ipykernel_9457/717144884.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Label[df.HostName=="192.168.56.171"] = 0
/var/folders/hg/c084_sjj0_1dhg3qjvswf_mh0000gn/T/ipykernel_9457/717144884.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Label_lifecycle[df.HostName=="192.168.56.171"] = 0


In [19]:
col_list = ['HostName', 'EventTemplate', 'EventTemplate2', 'Timestamp', 'Label', 'Label_lifecycle', 'PID_or_IP', 'Component']
one_hot_col_list = ['EventTemplate', 'EventTemplate2']
result_abs_path = os.path.join(root_folder, "modify")

df = pd.read_csv(os.path.join(result_abs_path, "NLP_label_syslog.csv"), usecols=col_list)
labels = df['Label'].unique()

df = pd.get_dummies(df, columns=one_hot_col_list)

if not os.path.exists(result_abs_path):
    os.makedirs(result_abs_path)
df.to_csv(os.path.join(result_abs_path, "relabel_syslog.csv"), index=False)

In [20]:
df = pd.read_csv(os.path.join(result_abs_path, "relabel_syslog.csv"))
# df = df[df.Component.str.contains('apache-error')]
df = df[df.HostName.isnull()]
# df = df[df.PID_or_IP.str.contains('^[0-9]+$')]
df.head(n=100)
# df.describe()
# df.PID_or_IP.unique()

/var/folders/hg/c084_sjj0_1dhg3qjvswf_mh0000gn/T/ipykernel_38144/1154463523.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(result_abs_path, "relabel_syslog.csv"))


,HostName,Component,PID_or_IP,Timestamp,Label,Label_lifecycle,"EventTemplate_""GET / HTTP/1.0"" 200 34990 ""-"" ""-""","EventTemplate_""GET / HTTP/1.1"" 200 34990 ""-"" ""Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)""","EventTemplate_""GET /.git/HEAD HTTP/1.1"" 404 465 ""-"" ""Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)""","EventTemplate_""GET /HNAP1 HTTP/1.1"" 404 461 ""-"" ""Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)""",...,EventTemplate_warning: TLS library problem: error:14076102:SSL routines:SSL23_GET_CLIENT_HELLO:unsupported protocol:s23_srvr.c:557:,EventTemplate_warning: TLS library problem: error:14094410:SSL routines:SSL3_READ_BYTES:sslv3 alert handshake failure:s3_pkt.c:1278:SSL alert number 40:,EventTemplate_warning: non-SMTP command from unknown[<*>]: yg:????,EventTemplate_warning: non-SMTP command from unknown[<*>]: {:?????????S????0???I,EventTemplate_warning: unknown[<*>]: SASL NTLM authentication failed: Invalid authentication mechanism,EventTemplate2_0,EventTemplate2_1,EventTemplate2_2,EventTemplate2_3,EventTemplate2_4
